# Exercise Notebook Part 2

**Practice exercises for Part 2** - Follows **Learning Notebook Part 2**

This notebook covers:
1. **Exercise 3**: TF-IDF from Scratch - IDF calculation and full TF-IDF (builds on Part 1's TF/BoW)
2. **Exercise 4**: Build TF-IDF Vectors from Scratch - Create complete TF-IDF vectors
3. **Exercise 5**: Similarity-Based Search with TF-IDF - Using cosine similarity for search
4. **Exercise 6**: Hybrid Search - Combining TF-IDF + keyword matching
5. **Exercise 7**: Compare Search Methods - Side-by-side comparison

**Important Pipeline Order:**
```
Preprocessing → Tokenization → Vectorization (BoW/TF → TF-IDF) → Similarity Search → Hybrid Search
```

**Instructions**: Complete each exercise by filling in the code cells marked with `# TODO`

**💡 Tip**: These exercises build on each other! Start with Exercise 3, then work through them in order. If you get stuck, refer back to the Learning Notebook Part 2 for guidance.

**Note**:
- **Prerequisites**: Complete **Exercise Notebook Part 1** first! You need to understand TF/BoW before learning TF-IDF.
- **Document clustering** is covered in **Learning Notebook Part 2** - no exercises needed here!
- Remember that TF-IDF is **syntactic** (word-based, no meaning). True semantic search (understanding meaning, synonyms) requires embeddings (Class 3)! **Semantic = meaning**.


In [11]:
# Install required packages if not already installed
# Uncomment the line below if gensim is not installed
# !pip install gensim

import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 35.4 MB/s eta 0:00:00


In [2]:
# Load movie data
# If running in Google Colab and data file doesn't exist, download it from GitHub
import os

if not os.path.exists('data/movies.csv'):
    print("Data file not found. Downloading from GitHub...")
    os.makedirs('data', exist_ok=True)
    import urllib.request
    url = 'https://raw.githubusercontent.com/samsung-ai-course/8th-9th-edition/main/Chapter%202%20-%20Natural%20Language%20Processing/Class%201%20%26%202%20-%20NLP%20and%20Search/data/movies.csv'
    urllib.request.urlretrieve(url, 'data/movies.csv')
    print("✓ Data file downloaded successfully!")

df = pd.read_csv('data/movies.csv')
print(f"Loaded {len(df)} movies")
df.head()


Data file not found. Downloading from GitHub...
✓ Data file downloaded successfully!
Loaded 10000 movies


,movie_id,title,description,genre,rating
0,1,Edge of Code,A compelling romance film about a young advent...,Romance,7.1
1,2,Storm of Secret,This captivating romance movie follows a quest...,Romance,6.3
2,3,Under Warrior Redux,"In this captivating war story, a secret organi...",War,7.3
3,4,Quest of Secret,A compelling fantasy film about a determined d...,Fantasy,8.3
4,5,Key of Game,A exploration adventure film about a master th...,Adventure,6.2


## Exercise 3: TF-IDF from Scratch

Implement TF-IDF calculation manually to understand how it works.

**Prerequisites**: You should have completed Exercise 1 in Part 1 where you learned TF (Term Frequency) / Bag of Words.

**Note**: This is labeled as Exercise 3 because it builds on Exercises 1-2 from Part 1.

**In this exercise, you'll learn:**
1. Calculate Inverse Document Frequency (IDF) - how rare a word is across all documents
2. Calculate TF-IDF = TF × IDF - weighted word importance
3. Build TF-IDF vectors from scratch

**Key Point**: TF-IDF improves on simple BoW by weighting words by their importance!


In [13]:
def calculate_tf(term, document_tokens):
    """
    Calculate Term Frequency: count(term) / total_terms

    Note: This is from Part 1! You should have implemented this already.
    If you haven't, complete it here first.

    Args:
        term (str): The word to count
        document_tokens (list): List of tokens in the document

    Returns:
        float: Term frequency
    """
    if len(document_tokens) == 0:
        return 0.0
    else:
      times = document_tokens.count(term)
      return times / len(document_tokens)

def calculate_idf(term, all_documents_tokens):
    """
    Calculate Inverse Document Frequency: log(total_docs / docs_containing_term)

    IDF measures how rare/important a word is:
    - High IDF = word appears in few documents (rare, informative)
    - Low IDF = word appears in many documents (common, less informative)

    Args:
        term (str): The word
        all_documents_tokens (list): List of documents, each is a list of tokens

    Returns:
        float: Inverse document frequency
    """
    import math

    total_docs = len(all_documents_tokens)

    # Count how many documents contain the term
    docs_with_term = sum(1 for doc in all_documents_tokens if term in doc)

    # Handle edge case: term doesn't appear in any document
    if docs_with_term == 0:
        return 0.0

    # Calculate log(total_docs / docs_with_term)
    return math.log(total_docs / docs_with_term)

def calculate_tfidf(term, document_tokens, all_documents_tokens):
    """
    Calculate TF-IDF = TF × IDF

    TF-IDF combines:
    - TF: How important is the word in THIS document?
    - IDF: How rare/important is the word across ALL documents?

    High TF-IDF = word is frequent in this document AND rare across documents (very informative!)

    Args:
        term (str): The word
        document_tokens (list): Tokens in current document
        all_documents_tokens (list): All documents as lists of tokens

    Returns:
        float: TF-IDF score
    """
    # Calculate TF and IDF, then multiply them
    tf = calculate_tf(term, document_tokens)
    idf = calculate_idf(term, all_documents_tokens)

    return tf * idf

# Test with simple example
# Note: Updated so "learning" has different TF in different docs to show different TF-IDF values
docs = [
    ["natural", "language", "processing"],
    ["machine", "learning", "natural"],           # "learning" appears 1/3 = 0.333
    ["deep", "learning", "learning", "language"]  # "learning" appears 2/4 = 0.500 (different TF!)
]

print("Testing TF-IDF calculation:")
print("=" * 60)
print(f"TF of 'natural' in doc 0: {calculate_tf('natural', docs[0])}")
print(f"IDF of 'natural': {calculate_idf('natural', docs)}")
print(f"TF-IDF of 'natural' in doc 0: {calculate_tfidf('natural', docs[0], docs)}")

print("\n" + "=" * 60)
print("Testing More Examples:")
print("=" * 60)

# Test different words
test_terms = ['natural', 'learning', 'processing', 'machine']
for term in test_terms:
    print(f"\nTerm: '{term}'")
    # Find which documents contain this term
    doc_indices = [i for i, doc in enumerate(docs) if term in doc]
    print(f"  Appears in documents: {doc_indices}")

    for doc_idx in doc_indices:
        tf = calculate_tf(term, docs[doc_idx])
        idf = calculate_idf(term, docs)
        tfidf = calculate_tfidf(term, docs[doc_idx], docs)
        print(f"  Doc {doc_idx}: TF={tf:.3f}, IDF={idf:.3f}, TF-IDF={tfidf:.3f}")

print("\n" + "=" * 60)
print("Key Insights:")
print("=" * 60)
print("1. Common words (like 'learning', 'natural') have lower IDF (appear in multiple docs)")
print("2. Rare words (like 'machine', 'processing') have higher IDF (appear in few docs)")
print("3. TF-IDF = TF × IDF balances document-specific importance with global rarity")
print("4. High TF-IDF = word is frequent in document AND rare across corpus (very informative!)")
print("5. Notice how 'learning' appears in 2 docs - same IDF (global rarity) but different TF-IDF due to different TF (document frequency)!")

print("\n💡 Reference: See Learning Notebook Part 2, cells 8-9 for detailed TF-IDF explanation")
print("💡 Think about edge cases: What if document is empty? Term not in any doc?")

Testing TF-IDF calculation:
TF of 'natural' in doc 0: 0.3333333333333333
IDF of 'natural': 0.4054651081081644
TF-IDF of 'natural' in doc 0: 0.13515503603605478

Testing More Examples:

Term: 'natural'
  Appears in documents: [0, 1]
  Doc 0: TF=0.333, IDF=0.405, TF-IDF=0.135
  Doc 1: TF=0.333, IDF=0.405, TF-IDF=0.135

Term: 'learning'
  Appears in documents: [1, 2]
  Doc 1: TF=0.333, IDF=0.405, TF-IDF=0.135
  Doc 2: TF=0.500, IDF=0.405, TF-IDF=0.203

Term: 'processing'
  Appears in documents: [0]
  Doc 0: TF=0.333, IDF=1.099, TF-IDF=0.366

Term: 'machine'
  Appears in documents: [1]
  Doc 1: TF=0.333, IDF=1.099, TF-IDF=0.366

Key Insights:
1. Common words (like 'learning', 'natural') have lower IDF (appear in multiple docs)
2. Rare words (like 'machine', 'processing') have higher IDF (appear in few docs)
3. TF-IDF = TF × IDF balances document-specific importance with global rarity
4. High TF-IDF = word is frequent in document AND rare across corpus (very informative!)
5. Notice how 'lea

## Exercise 4: Build TF-IDF Vectors from Scratch

After calculating individual TF-IDF scores, build complete TF-IDF vectors for documents.

**Goal**: Create TF-IDF vectors (like BoW vectors but with TF-IDF scores instead of counts).

**What you'll implement:**
1. Build vocabulary from all documents
2. For each document, calculate TF-IDF for each word in vocabulary
3. Create a vector where each position corresponds to a word in vocabulary
4. Return the TF-IDF vector


In [4]:
def create_tfidf_vector(document_tokens, vocabulary, all_documents_tokens):
    """
    Create a TF-IDF vector for a document.

    Similar to BoW vector, but uses TF-IDF scores instead of counts!

    Args:
        document_tokens: List of tokens in the document
        vocabulary: List of all unique words in the corpus (sorted)
        all_documents_tokens: List of all documents as lists of tokens

    Returns:
        list: TF-IDF vector where each element is the TF-IDF score for that word
    """
    vector = []

    # Loop through vocabulary and calculate TF-IDF for each word
    for word in vocabulary:
        # Calculate TF-IDF score for that word in this document
        # If word doesn't appear in document, TF-IDF = 0 (handled by calculate_tfidf)
        tfidf_score = calculate_tfidf(word, document_tokens, all_documents_tokens)
        vector.append(tfidf_score)

    return vector

def create_tfidf_matrix(all_documents_tokens):
    """
    Create a TF-IDF matrix for all documents.

    Args:
        all_documents_tokens: List of documents, each is a list of tokens

    Returns:
        tuple: (tfidf_matrix, vocabulary)
            - tfidf_matrix: List of TF-IDF vectors (one per document)
            - vocabulary: List of all unique words (sorted)
    """
    # Step 1 - Build vocabulary from all documents
    # Build vocabulary
    all_words = set()
    for doc_tokens in all_documents_tokens:
        all_words.update(doc_tokens)
    vocabulary = sorted(list(all_words))

    # Step 2 - For each document, create TF-IDF vector
    matrix = []
    for doc_tokens in all_documents_tokens:
        vector = create_tfidf_vector(doc_tokens, vocabulary, all_documents_tokens)
        matrix.append(vector)

    # Step 3 - Return matrix and vocabulary
    return matrix, vocabulary

# Test with simple example
test_docs = [
    ["natural", "language", "processing"],
    ["machine", "learning", "natural"],
    ["deep", "learning", "language"]
]

print("Testing TF-IDF Vector Creation:")
print("=" * 60)

tfidf_matrix, vocab = create_tfidf_matrix(test_docs)

print(f"Vocabulary: {vocab}")
print(f"\nTF-IDF Matrix:")
print(f"Shape: {len(tfidf_matrix)} documents × {len(vocab)} words")

for i, doc_vector in enumerate(tfidf_matrix):
    print(f"\nDocument {i}: {test_docs[i]}")
    print(f"TF-IDF Vector: {doc_vector}")

# Compare with BoW (from Part 1)
print("\n" + "=" * 60)
print("Comparison: BoW vs TF-IDF")
print("=" * 60)

# Simple BoW for comparison
def create_bow_vector_simple(doc_tokens, vocab):
    word_counts = Counter(doc_tokens)
    return [word_counts.get(word, 0) for word in vocab]

print("\nBoW vectors (word counts):")
for i, doc_tokens in enumerate(test_docs):
    bow_vec = create_bow_vector_simple(doc_tokens, vocab)
    print(f"Doc {i}: {bow_vec}")

print("\nTF-IDF vectors (weighted scores):")
for i, doc_vector in enumerate(tfidf_matrix):
    print(f"Doc {i}: {[round(x, 3) for x in doc_vector]}")

print("\n💡 Key Difference:")
print("  - BoW: Simple counts (0, 1, 2, ...)")
print("  - TF-IDF: Weighted scores (0.0 to higher values)")
print("  - TF-IDF emphasizes rare, informative words more!")


Testing TF-IDF Vector Creation:
Vocabulary: ['deep', 'language', 'learning', 'machine', 'natural', 'processing']

TF-IDF Matrix:
Shape: 3 documents × 6 words

Document 0: ['natural', 'language', 'processing']
TF-IDF Vector: [0.0, 0.13515503603605478, 0.0, 0.0, 0.13515503603605478, 0.3662040962227032]

Document 1: ['machine', 'learning', 'natural']
TF-IDF Vector: [0.0, 0.0, 0.13515503603605478, 0.3662040962227032, 0.13515503603605478, 0.0]

Document 2: ['deep', 'learning', 'language']
TF-IDF Vector: [0.3662040962227032, 0.13515503603605478, 0.13515503603605478, 0.0, 0.0, 0.0]

Comparison: BoW vs TF-IDF

BoW vectors (word counts):
Doc 0: [0, 1, 0, 0, 1, 1]
Doc 1: [0, 0, 1, 1, 1, 0]
Doc 2: [1, 1, 1, 0, 0, 0]

TF-IDF vectors (weighted scores):
Doc 0: [0.0, 0.135, 0.0, 0.0, 0.135, 0.366]
Doc 1: [0.0, 0.0, 0.135, 0.366, 0.135, 0.0]
Doc 2: [0.366, 0.135, 0.135, 0.0, 0.0, 0.0]

💡 Key Difference:
  - BoW: Simple counts (0, 1, 2, ...)
  - TF-IDF: Weighted scores (0.0 to higher values)
  - TF-IDF

## Exercise 5: Similarity-Based Search with TF-IDF

Implement similarity-based search using TF-IDF vectors and cosine similarity.

**Prerequisites**: You need TF-IDF vectors! Use scikit-learn's `TfidfVectorizer` (or build from scratch using Exercise 3 & 4).

**What you'll implement:**
1. Convert query to TF-IDF vector
2. Calculate cosine similarity between query and all documents
3. Return top-k most similar documents

**Note**: This is often called "semantic search" but TF-IDF is still fundamentally keyword-based. True semantic search (understanding synonyms and meaning) requires embeddings (Class 3)!


In [5]:
def cosine_similarity_manual(vec1, vec2):
    """
    Calculate cosine similarity between two vectors manually.

    Cosine Similarity = (A · B) / (||A|| × ||B||)
    - A · B = dot product
    - ||A|| = magnitude (L2 norm) of vector A

    Args:
        vec1: First vector (list or array)
        vec2: Second vector (list or array)

    Returns:
        float: Cosine similarity (range: -1 to 1, but for TF-IDF usually 0 to 1)
    """
    import numpy as np

    # Convert to numpy arrays
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    # Calculate dot product (A · B)
    dot_product = np.dot(vec1, vec2)

    # Calculate magnitudes (||A|| and ||B||)
    magnitude1 = np.linalg.norm(vec1)
    magnitude2 = np.linalg.norm(vec2)

    # Handle division by zero if both vectors are zero
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0

    # Return cosine similarity = dot_product / (magnitude1 * magnitude2)
    return dot_product / (magnitude1 * magnitude2)

def search_tfidf(query, vectorizer, tfidf_vectors, df, top_k=5):
    """
    Similarity-based search using TF-IDF and cosine similarity.

    Steps:
    1. Convert query to TF-IDF vector using the same vectorizer
    2. Calculate cosine similarity with all document vectors
    3. Get top_k most similar documents
    4. Return results as DataFrame

    Args:
        query: Search query string
        vectorizer: Fitted TfidfVectorizer
        tfidf_vectors: TF-IDF matrix for all documents (sparse matrix)
        df: DataFrame with movie data
        top_k: Number of results to return

    Returns:
        DataFrame with search results (title, similarity, description)
    """
    # Step 1 - Convert query to TF-IDF vector
    # Use vectorizer.transform([query]), NOT fit_transform()
    # Why? Because vectorizer is already fitted on the corpus!
    query_vector = vectorizer.transform([query])

    # Step 2 - Calculate cosine similarity with all documents
    # Use cosine_similarity from sklearn
    # Remember to get the first row [0] since query_vector is 1xN
    similarities = cosine_similarity(query_vector, tfidf_vectors)[0]

    # Step 3 - Get indices of top_k most similar documents
    # Use argsort() - it returns indices in ascending order
    # You want descending (highest similarity first), so reverse it!
    top_indices = similarities.argsort()[-top_k:][::-1]

    # Step 4 - Build results DataFrame with movie_id, title, similarity, description
    results = []
    for idx in top_indices:
        results.append({
            'movie_id': df.iloc[idx]['movie_id'],
            'title': df.iloc[idx]['title'],
            'similarity': similarities[idx],
            'description': df.iloc[idx]['description']
        })

    return pd.DataFrame(results)

# First, create TF-IDF vectors for the movie corpus
print("Creating TF-IDF vectors...")
vectorizer = TfidfVectorizer(max_features=100, stop_words='english', lowercase=True)
tfidf_vectors = vectorizer.fit_transform(df['description'])
print(f"✓ Created TF-IDF matrix: {tfidf_vectors.shape}")

# Test your search function
print("\n" + "=" * 70)
print("Testing Similarity Search:")
print("=" * 70)

test_query = "space adventure exploration"
print(f"\nQuery: '{test_query}'")

results = search_tfidf(test_query, vectorizer, tfidf_vectors, df, top_k=5)

if len(results) > 0:
    print("\nTop Results:")
    print(results[['title', 'similarity']].to_string(index=False))
    print("\n✅ Success! Your search function is working.")
else:
    print("\n⚠️  Results empty - implement the function first!")

print("\n💡 Key Points:")
print("  - Cosine similarity measures the angle between vectors")
print("  - Range: -1 to 1 (but for TF-IDF, usually 0 to 1)")
print("  - Higher similarity = more similar documents")
print("  - Similarity search ranks by relevance (better than keyword search!)")

print("\n💡 Reference: See Learning Notebook Part 2, cell 11 for working example")

Creating TF-IDF vectors...
✓ Created TF-IDF matrix: (10000, 100)

Testing Similarity Search:

Query: 'space adventure exploration'

Top Results:
            title  similarity
Under Point Rises    0.688765
  Out of Ice Tale    0.649141
      Wind of Ice    0.599094
 Guardian of Love    0.593995
      Dream: Line    0.554876

✅ Success! Your search function is working.

💡 Key Points:
  - Cosine similarity measures the angle between vectors
  - Range: -1 to 1 (but for TF-IDF, usually 0 to 1)
  - Higher similarity = more similar documents
  - Similarity search ranks by relevance (better than keyword search!)

💡 Reference: See Learning Notebook Part 2, cell 11 for working example


In [6]:
def euclidean_distance_manual(vec1, vec2):
    """
    Calculate Euclidean distance between two vectors manually.

    Euclidean Distance = √(Σ(a_i - b_i)²)
    - Sum of squared differences
    - Square root of the sum

    Args:
        vec1: First vector (list or array)
        vec2: Second vector (list or array)

    Returns:
        float: Euclidean distance (range: 0 to ∞, where 0 = identical)
    """
    import numpy as np

    # Convert to numpy arrays
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    # Calculate Euclidean distance
    # Formula is √(Σ(a_i - b_i)²)
    diff = vec1 - vec2
    squared_diff = diff ** 2
    sum_squared = np.sum(squared_diff)
    distance = np.sqrt(sum_squared)

    return distance

def search_euclidean(query, vectorizer, tfidf_vectors, df, top_k=5):
    """
    Similarity-based search using TF-IDF and Euclidean distance.

    Steps:
    1. Convert query to TF-IDF vector
    2. Calculate Euclidean distance with all documents
    3. Get top_k CLOSEST documents (smaller distance = more similar!)
    4. Return results as DataFrame

    Args:
        query: Search query string
        vectorizer: Fitted TfidfVectorizer
        tfidf_vectors: TF-IDF matrix for all documents
        df: DataFrame with movie data
        top_k: Number of results to return

    Returns:
        DataFrame with search results (title, distance, description)
    """
    from sklearn.metrics.pairwise import euclidean_distances

    # Step 1 - Convert query to TF-IDF vector
    # Similar to Exercise 5, use vectorizer.transform()
    query_vector = vectorizer.transform([query])

    # Step 2 - Calculate Euclidean distance with all documents
    # Use euclidean_distances from sklearn (or your manual function)
    distances = euclidean_distances(query_vector, tfidf_vectors)[0]

    # Step 3 - Get indices of top_k SMALLEST distances
    # argsort() returns indices in ascending order - perfect for Euclidean!
    # Smaller distance = more similar
    top_indices = distances.argsort()[:top_k]

    # Step 4 - Build results DataFrame with movie_id, title, distance, description
    results = []
    for idx in top_indices:
        results.append({
            'movie_id': df.iloc[idx]['movie_id'],
            'title': df.iloc[idx]['title'],
            'distance': distances[idx],
            'description': df.iloc[idx]['description']
        })

    return pd.DataFrame(results)

# Test your Euclidean distance search function
print("\n" + "=" * 70)
print("Testing Euclidean Distance Search:")
print("=" * 70)

test_query = "space adventure exploration"
print(f"\nQuery: '{test_query}'")

results_euclidean = search_euclidean(test_query, vectorizer, tfidf_vectors, df, top_k=5)

if len(results_euclidean) > 0:
    print("\nTop Results (Euclidean Distance):")
    print(results_euclidean[['title', 'distance']].to_string(index=False))
    print("\n💡 Remember: Smaller distance = more similar!")
else:
    print("\n⚠️  Results empty - implement the function first!")

# Compare with cosine similarity results
print("\n" + "=" * 70)
print("Comparing Cosine vs Euclidean:")
print("=" * 70)

results_cosine = search_tfidf(test_query, vectorizer, tfidf_vectors, df, top_k=5)

if len(results_cosine) > 0 and len(results_euclidean) > 0:
    print("\nCosine Similarity Results:")
    print(results_cosine[['title', 'similarity']].to_string(index=False))

    print("\nEuclidean Distance Results:")
    print(results_euclidean[['title', 'distance']].to_string(index=False))

    # Check overlap
    cosine_titles = set(results_cosine['title'])
    euclidean_titles = set(results_euclidean['title'])
    overlap = cosine_titles.intersection(euclidean_titles)

    print(f"\n📊 Overlap: {len(overlap)}/5 movies appear in both top-5 lists")
    if overlap:
        print(f"   Common movies: {', '.join(list(overlap)[:3])}...")

    print("\n💡 Key Observations:")
    print("  - Cosine: Measures direction/pattern similarity")
    print("  - Euclidean: Measures absolute distance in space")
    print("  - For TF-IDF text search, cosine is usually better!")
else:
    print("\n⚠️  Implement both search functions to see the comparison!")

print("\n💡 Reference: See Learning Notebook Part 2 for Cosine vs Euclidean comparison")


Testing Euclidean Distance Search:

Query: 'space adventure exploration'

Top Results (Euclidean Distance):
            title  distance
Under Point Rises  0.788968
  Out of Ice Tale  0.837686
      Wind of Ice  0.895440
 Guardian of Love  0.901116
      Dream: Line  0.943530

💡 Remember: Smaller distance = more similar!

Comparing Cosine vs Euclidean:

Cosine Similarity Results:
            title  similarity
Under Point Rises    0.688765
  Out of Ice Tale    0.649141
      Wind of Ice    0.599094
 Guardian of Love    0.593995
      Dream: Line    0.554876

Euclidean Distance Results:
            title  distance
Under Point Rises  0.788968
  Out of Ice Tale  0.837686
      Wind of Ice  0.895440
 Guardian of Love  0.901116
      Dream: Line  0.943530

📊 Overlap: 5/5 movies appear in both top-5 lists
   Common movies: Dream: Line, Out of Ice Tale, Guardian of Love...

💡 Key Observations:
  - Cosine: Measures direction/pattern similarity
  - Euclidean: Measures absolute distance in space


## Exercise 5b: Euclidean Distance Search (Bonus)

Now implement the same search functionality but using **Euclidean Distance** instead of Cosine Similarity!

**Goal**: Understand the difference between cosine and Euclidean distance metrics.

**Key Differences**:
- **Cosine Similarity**: Measures direction/angle (ignores magnitude) - Range: -1 to 1
- **Euclidean Distance**: Measures absolute distance (considers magnitude) - Range: 0 to ∞

**For search results**:
- **Cosine**: Higher similarity = more similar (sort descending)
- **Euclidean**: Smaller distance = more similar (sort ascending)

**Your Task**: Implement `search_euclidean()` function that:
1. Converts query to TF-IDF vector
2. Calculates Euclidean distance to all documents
3. Returns documents with **smallest distance** (most similar)

## Exercise 6: Hybrid Search

**Goal**: Combine TF-IDF similarity search with keyword matching for the best of both worlds!

**What you'll implement:**
1. Get TF-IDF similarity scores
2. Get keyword match scores
3. Combine scores using weighted sum
4. Add boost for exact keyword matches
5. Return top-k results sorted by combined score

**Key Benefits:**
- Precision from keyword matching (exact matches)
- Recall from TF-IDF (related content)
- Most robust approach for production systems!


In [7]:
# Hybrid Search Implementation - Combining TF-IDF + Keyword Matching
def hybrid_search(query, vectorizer, tfidf_vectors, df, top_k=5, keyword_weight=0.3, boost_factor=0.2):
    """
    Hybrid search combining TF-IDF similarity and keyword matching

    Args:
        query: Search query string
        vectorizer: Fitted TF-IDF vectorizer
        tfidf_vectors: TF-IDF vectors for all documents
        df: DataFrame with movie data
        top_k: Number of results to return
        keyword_weight: Weight for keyword matching (0-1). 0.3 = 30% keyword, 70% TF-IDF
        boost_factor: Additional boost for exact keyword matches (added to TF-IDF score)

    Returns:
        DataFrame with search results sorted by combined score
    """
    import re

    # Step 1: Get TF-IDF similarity scores
    # Convert query to TF-IDF vector and calculate cosine similarity with all documents
    query_vector = vectorizer.transform([query])
    tfidf_similarities = cosine_similarity(query_vector, tfidf_vectors)[0]

    # Step 2: Get keyword match scores
    # Extract words from query and count how many appear in each document
    query_words = set(re.findall(r'\w+', query.lower()))  # Extract words from query
    keyword_scores = []
    keyword_match_counts = []

    for idx, row in df.iterrows():
        text = str(row['description']).lower()
        text_words = set(re.findall(r'\w+', text))

        # Count how many query words appear in document using set intersection
        matches = query_words.intersection(text_words)
        match_count = len(matches)

        # Calculate normalized match ratio (0 to 1)
        match_ratio = match_count / len(query_words) if len(query_words) > 0 else 0

        keyword_scores.append(match_ratio)  # Normalized: 0 to 1
        keyword_match_counts.append(match_count)  # Raw count

    keyword_scores = np.array(keyword_scores)
    keyword_match_counts = np.array(keyword_match_counts)

    # Step 3: Combine scores using weighted sum
    # Normalize TF-IDF scores to 0-1 range using min-max normalization
    # This ensures both keyword and TF-IDF scores are on the same scale
    tfidf_normalized = (tfidf_similarities - tfidf_similarities.min()) / (tfidf_similarities.max() - tfidf_similarities.min() + 1e-8)

    # TODO: Combine scores using weighted sum
    # Formula: combined = (keyword_weight × keyword_score) + ((1 - keyword_weight) × tfidf_score)
    # Example: If keyword_weight=0.3, then 30% keyword + 70% TF-IDF
    # Hint: Use numpy array operations for element-wise multiplication
    combined_scores = (keyword_weight * keyword_scores) + ((1 - keyword_weight) * tfidf_normalized)

    # Step 4: Add boost for exact keyword matches
    # Documents with more keyword matches get an additional boost
    # This helps prioritize exact matches while still benefiting from TF-IDF ranking
    max_matches = keyword_match_counts.max() if keyword_match_counts.max() > 0 else 1
    keyword_boost = (keyword_match_counts / max_matches) * boost_factor
    final_scores = combined_scores + keyword_boost

    # Step 5: Get top_k results
    # Sort by final scores (descending) and get top_k indices
    top_indices = final_scores.argsort()[-top_k:][::-1]

    # Build results DataFrame with all relevant information
    results = []
    for idx in top_indices:
        results.append({
            'movie_id': df.iloc[idx]['movie_id'],
            'title': df.iloc[idx]['title'],
            'final_score': final_scores[idx],
            'tfidf_score': tfidf_similarities[idx],
            'keyword_score': keyword_scores[idx],
            'keyword_matches': keyword_match_counts[idx],
            'description': df.iloc[idx]['description']
        })

    return pd.DataFrame(results)


## Exercise 7: Compare Search Methods

**Goal**: Compare keyword search, TF-IDF search, and hybrid search side-by-side!

**Your Task**: Test all three search methods on the same query and observe the differences.

**This helps you understand:**
- When to use each method
- How different approaches affect results
- Why hybrid search is often the best choice


In [8]:
# TODO: Compare all three search methods on the same query

def simple_keyword_search_compare(query, df, top_k=5):
    """
    Simple keyword search for comparison (from Part 1).
    Finds documents containing query words.
    """
    import re
    query_words = set(re.findall(r'\w+', query.lower()))
    results = []

    for idx, row in df.iterrows():
        text = str(row['description']).lower()
        text_words = set(re.findall(r'\w+', text))
        matches = query_words.intersection(text_words)
        match_count = len(matches)

        if match_count > 0:
            results.append({
                'movie_id': row['movie_id'],
                'title': row['title'],
                'match_count': match_count,
                'description': row['description']
            })

    results_df = pd.DataFrame(results)
    if len(results_df) > 0:
        results_df = results_df.sort_values('match_count', ascending=False).head(top_k)
    return results_df

# Test query
test_query = "space adventure aliens"

print("=" * 80)
print(f"COMPARISON: Search Results for Query '{test_query}'")
print("=" * 80)

# Method 1: Keyword Search
print("\n" + "=" * 80)
print("1. KEYWORD SEARCH (Exact Word Matching)")
print("=" * 80)
keyword_results = simple_keyword_search_compare(test_query, df, top_k=5)
if len(keyword_results) > 0:
    print(keyword_results[['title', 'match_count']].to_string(index=False))
    print("\n✅ Finds documents with exact word matches")
    print("❌ No ranking beyond match count")
    print("❌ Misses related content without exact words")
else:
    print("No results found")

# Method 2: TF-IDF Search
print("\n" + "=" * 80)
print("2. TF-IDF SIMILARITY SEARCH")
print("=" * 80)
tfidf_results = search_tfidf(test_query, vectorizer, tfidf_vectors, df, top_k=5)
if len(tfidf_results) > 0:
    print(tfidf_results[['title', 'similarity']].to_string(index=False))
    print("\n✅ Ranks by importance (TF-IDF weighted)")
    print("✅ Finds related content even without exact matches")
    print("⚠️  Might rank exact matches lower if they have low TF-IDF")
else:
    print("⚠️  Implement search_tfidf first!")

# Method 3: Hybrid Search
print("\n" + "=" * 80)
print("3. HYBRID SEARCH (TF-IDF + Keyword Matching)")
print("=" * 80)
hybrid_results = hybrid_search(test_query, vectorizer, tfidf_vectors, df, top_k=5,
                               keyword_weight=0.3, boost_factor=0.2)
if len(hybrid_results) > 0:
    print(hybrid_results[['title', 'final_score']].head(5).to_string(index=False))
    print("\n✅ Combines best of both: keyword precision + TF-IDF recall")
    print("✅ Boosts exact matches while still finding related content")
    print("✅ Most flexible and robust approach")
else:
    print("⚠️  Implement hybrid_search first!")

print("\n" + "=" * 80)
print("SUMMARY: When to Use Each Method")
print("=" * 80)
print("""
Keyword Search:
  - ✅ Fast and simple
  - ✅ Good for exact phrase matching
  - ❌ Too rigid, misses related content
  - Use when: You need exact matches only

TF-IDF Search:
  - ✅ Better ranking by importance
  - ✅ Finds related content
  - ⚠️  Might miss exact matches
  - Use when: You want semantic-like matching (but still keyword-based)

Hybrid Search:
  - ✅ Best of both worlds
  - ✅ Precision from keywords + recall from TF-IDF
  - ✅ Most robust for production systems
  - Use when: Building a real search system (recommended!)
""")

print("💡 For our movie search system, hybrid search gives the best results!")
print("💡 Try different queries and see how each method performs!")


COMPARISON: Search Results for Query 'space adventure aliens'

1. KEYWORD SEARCH (Exact Word Matching)
              title  match_count
Through Hope Reborn            2
      Hope of Dream            2
       Out of Peace            2
     The Gate Redux            2
     Shadow of Game            2

✅ Finds documents with exact word matches
❌ No ranking beyond match count
❌ Misses related content without exact words

2. TF-IDF SIMILARITY SEARCH
            title  similarity
Under Point Rises    0.688765
  Out of Ice Tale    0.649141
      Wind of Ice    0.599094
 Guardian of Love    0.593995
      Dream: Line    0.554876

✅ Ranks by importance (TF-IDF weighted)
✅ Finds related content even without exact matches
⚠️  Might rank exact matches lower if they have low TF-IDF

3. HYBRID SEARCH (TF-IDF + Keyword Matching)
                  title  final_score
      Under Point Rises     1.100000
        Out of Ice Tale     1.059730
       Guardian of Love     1.003685
            Dream: Line  

## Bonus Exercise: Evaluate and Compare Search Methods

**Goal**: Use your metrics to objectively compare different search approaches!

**Your task:**
1. Create a function that runs a search query and returns top-K results
2. Define ground truth (relevant movie IDs) for a test query
3. Run the same query through:
   - Keyword search
   - TF-IDF similarity search
   - Hybrid search (with different parameters)
4. Calculate Precision, Recall, NDCG, and MRR for each
5. Determine which approach works best!

**This is what real search engineers do** - use metrics to guide improvements!

In [9]:
# Bonus Exercise: Compare Search Methods with Metrics

def evaluate_search_method(search_function, query, relevant_ids, k=10):
    """
    Evaluate a search method using multiple metrics.

    Args:
        search_function: Function that takes query and returns ranked results
        query: Search query string
        relevant_ids: Ground truth relevant IDs
        k: Number of top results to evaluate

    Returns:
        dict: Dictionary of metric scores
    """
    # TODO: Call search_function to get results
    # TODO: Calculate Precision@K, Recall@K, NDCG@K, RR
    # TODO: Return as dictionary

    # Hint: Structure of return value:
    # return {
    #     'precision@k': ...,
    #     'recall@k': ...,
    #     'ndcg@k': ...,
    #     'mrr': ...
    # }

    pass  # Remove this and implement

# Example usage (after implementing):
print("="*70)
print("Comparing Search Methods")
print("="*70)

query = "space adventure exploration"

# You would need to define:
# 1. Ground truth relevant movies for this query
# 2. Functions for each search method

# Example:
# ground_truth = [10, 25, 42, 67, 89, 103, 156]  # Movies actually about space adventure
#
# metrics_keyword = evaluate_search_method(keyword_search, query, ground_truth)
# metrics_tfidf = evaluate_search_method(tfidf_search, query, ground_truth)
# metrics_hybrid = evaluate_search_method(hybrid_search, query, ground_truth)
#
# # Compare results
# print(f"\nKeyword Search:  P={metrics_keyword['precision@k']:.3f} NDCG={metrics_keyword['ndcg@k']:.3f}")
# print(f"TF-IDF Search:   P={metrics_tfidf['precision@k']:.3f} NDCG={metrics_tfidf['ndcg@k']:.3f}")
# print(f"Hybrid Search:   P={metrics_hybrid['precision@k']:.3f} NDCG={metrics_hybrid['ndcg@k']:.3f}")

print("\n💡 This is how you tune search systems in production!")
print("   Try different parameters, measure with metrics, pick the best!")
print("\n📚 Reference: Learning Notebook Part 2 for complete examples")

Comparing Search Methods

💡 This is how you tune search systems in production!
   Try different parameters, measure with metrics, pick the best!

📚 Reference: Learning Notebook Part 2 for complete examples


## Exercise 5: Implement Precision@K and Recall@K

**Goal**: Implement the two most fundamental search evaluation metrics from scratch!

**Prerequisites**: Complete Exercise 3 (similarity search) first - you'll need search results to evaluate.

**What you'll implement:**
1. `precision_at_k()` - Calculate what % of top-K results are relevant
2. `recall_at_k()` - Calculate what % of all relevant docs are found in top-K
3. Test both metrics on movie search results
4. Understand the precision-recall trade-off

**💡 Reference**: See Learning Notebook Part 2 - Evaluation Metrics section for detailed explanations!

In [14]:
# Exercise 8: Implement Precision@K and Recall@K

from collections import Counter  # se ainda não estiver importado

def precision_at_k(relevant_ids, search_results, k):
    """
    Calculate Precision@K.
    """
    # Se K for 0, evitamos divisão por zero
    if k == 0:
        return 0.0

    # Converter para set para procurar mais rápido
    relevant_set = set(relevant_ids)

    # Top-K resultados
    top_k = search_results[:k]

    # Contar quantos do top-K são relevantes
    relevant_in_top_k = sum(1 for doc_id in top_k if doc_id in relevant_set)

    # Precision@K = relevantes no top-K / K
    return relevant_in_top_k / k


def recall_at_k(relevant_ids, search_results, k):
    """
    Calculate Recall@K.
    """
    total_relevant = len(relevant_ids)

    # Se não houver documentos relevantes, definimos recall como 0.0
    if total_relevant == 0:
        return 0.0

    relevant_set = set(relevant_ids)
    top_k = search_results[:k]

    # Contar quantos relevantes aparecem no top-K
    relevant_in_top_k = sum(1 for doc_id in top_k if doc_id in relevant_set)

    # Recall@K = relevantes encontrados no top-K / total de relevantes
    return relevant_in_top_k / total_relevant


# Test with example data
# Ground truth: these are the truly relevant movies for query "space adventure"
relevant_movie_ids = [10, 25, 42, 67, 89, 103, 156, 201]  # 8 relevant total

# Simulated search results (ranked by some search algorithm)
search_results = [10, 15, 25, 30, 42, 55, 67, 70, 80, 89]  # Top 10 returned
# Relevant in results: 10, 25, 42, 67, 89 (5 out of 10)

print("Testing your implementations:")
print("="*60)
print(f"Ground truth: {len(relevant_movie_ids)} relevant movies")
print(f"Search results: {len(search_results)} returned\n")

# Test at different K values
for k in [1, 3, 5, 10]:
    prec = precision_at_k(relevant_movie_ids, search_results, k)
    rec = recall_at_k(relevant_movie_ids, search_results, k)
    print(f"K={k}:")
    print(f"  Precision@{k}: {prec:.3f} ({prec*100:.1f}%)")
    print(f"  Recall@{k}:    {rec:.3f} ({rec*100:.1f}%)\n")

# Visualize the precision-recall trade-off
print("="*60)
print("Precision-Recall Trade-off:")
print("="*60)
print("Notice what happens as K increases:")
print("  - Precision often decreases (more noise)")
print("  - Recall increases (finding more relevant docs)")
print("\nThis is the fundamental trade-off in search systems!")

# Expected output (if implemented correctly):
# K=1:  Precision=1.0 (100%), Recall=0.125 (12.5%)
# K=10: Precision=0.5 (50%), Recall=0.625 (62.5%)

print("\n💡 Tips:")
print("  1. Precision measures quality: 'Are the results I show relevant?'")
print("  2. Recall measures completeness: 'Did I find all relevant results?'")
print("  3. You usually can't maximize both at the same time!")
print("\n📚 Reference: Learning Notebook Part 2, Evaluation Metrics section")

Testing your implementations:
Ground truth: 8 relevant movies
Search results: 10 returned

K=1:
  Precision@1: 1.000 (100.0%)
  Recall@1:    0.125 (12.5%)

K=3:
  Precision@3: 0.667 (66.7%)
  Recall@3:    0.250 (25.0%)

K=5:
  Precision@5: 0.600 (60.0%)
  Recall@5:    0.375 (37.5%)

K=10:
  Precision@10: 0.500 (50.0%)
  Recall@10:    0.625 (62.5%)

Precision-Recall Trade-off:
Notice what happens as K increases:
  - Precision often decreases (more noise)
  - Recall increases (finding more relevant docs)

This is the fundamental trade-off in search systems!

💡 Tips:
  1. Precision measures quality: 'Are the results I show relevant?'
  2. Recall measures completeness: 'Did I find all relevant results?'
  3. You usually can't maximize both at the same time!

📚 Reference: Learning Notebook Part 2, Evaluation Metrics section


## Exercise 6: Implement MRR (Mean Reciprocal Rank)

**Goal**: Implement MRR - a metric that measures how quickly users find a relevant result!

**What you'll implement:**
1. `reciprocal_rank()` - Calculate RR for a single query (1 / position of first relevant result)
2. `mean_reciprocal_rank()` - Calculate MRR across multiple queries
3. Understand when MRR is the right metric to use

**Key Insight**: MRR is perfect for question-answering scenarios where users only need ONE good answer!

**💡 Reference**: Learning Notebook Part 2, MRR section

In [17]:
# Exercise 9: Implement MRR
import numpy as np

def reciprocal_rank(relevant_ids, search_results):
    """
    Calculate Reciprocal Rank for a single query.
    """
    relevant_set = set(relevant_ids)  # para lookup mais rápido

    # percorrer resultados com posição a começar em 1
    for position, doc_id in enumerate(search_results, start=1):
        if doc_id in relevant_set:
            return 1.0 / position  # RR = 1 / posição do primeiro relevante

    # se não houver nenhum relevante nos resultados
    return 0.0


def mean_reciprocal_rank(relevant_ids_list, search_results_list):
    """
    Calculate Mean Reciprocal Rank across multiple queries.
    """
    rr_scores = []

    for rel_ids, results in zip(relevant_ids_list, search_results_list):
        rr = reciprocal_rank(rel_ids, results)
        rr_scores.append(rr)

    if len(rr_scores) == 0:
        return 0.0

    return float(np.mean(rr_scores))


# Test with example data
print("Testing Reciprocal Rank:")
print("="*60)

# Single query example
relevant = [10, 25, 42]
results = [5, 10, 15, 25, 42]  # First relevant at position 2
rr = reciprocal_rank(relevant, results)
print(f"Search results: {results}")
print(f"Relevant IDs: {relevant}")
print(f"Reciprocal Rank: {rr:.3f}")
print(f"First relevant result at position: {1/rr:.0f}" if rr > 0 else "No relevant result")

# Multiple queries example
print("\n" + "="*60)
print("Testing Mean Reciprocal Rank (3 queries):")
print("="*60)

queries_relevant = [
    [10, 25, 42],       # Query 1 relevant IDs
    [5, 15, 30],        # Query 2 relevant IDs
    [1, 2, 3]           # Query 3 relevant IDs
]

queries_results = [
    [10, 11, 12, 13],   # Query 1: first relevant at position 1 → RR=1.0
    [20, 21, 15, 22],   # Query 2: first relevant at position 3 → RR=0.333
    [50, 51, 52, 1],    # Query 3: first relevant at position 4 → RR=0.25
]

for i, (rel, res) in enumerate(zip(queries_relevant, queries_results), 1):
    rr = reciprocal_rank(rel, res)
    pos = int(1/rr) if rr > 0 else "N/A"
    print(f"Query {i}: RR={rr:.3f} (first relevant at position {pos})")

mrr = mean_reciprocal_rank(queries_relevant, queries_results)
print(f"\nMean Reciprocal Rank: {mrr:.3f}")

# Expected output (if implemented correctly):
# Query 1: RR=1.0, Query 2: RR=0.333, Query 3: RR=0.25
# MRR = (1.0 + 0.333 + 0.25) / 3 = 0.528

print("\n" + "="*60)
print("💡 When to use MRR:")
print("  ✅ Question answering ('What is the capital of France?')")
print("  ✅ Navigational search (user looking for a specific page)")
print("  ✅ When users only need ONE good result")
print("  ❌ NOT good when users need multiple relevant results")
print("\n📚 Reference: Learning Notebook Part 2, MRR section")

Testing Reciprocal Rank:
Search results: [5, 10, 15, 25, 42]
Relevant IDs: [10, 25, 42]
Reciprocal Rank: 0.500
First relevant result at position: 2

Testing Mean Reciprocal Rank (3 queries):
Query 1: RR=1.000 (first relevant at position 1)
Query 2: RR=0.333 (first relevant at position 3)
Query 3: RR=0.250 (first relevant at position 4)

Mean Reciprocal Rank: 0.528

💡 When to use MRR:
  ✅ Question answering ('What is the capital of France?')
  ✅ Navigational search (user looking for a specific page)
  ✅ When users only need ONE good result
  ❌ NOT good when users need multiple relevant results

📚 Reference: Learning Notebook Part 2, MRR section


## Exercise 7: Implement NDCG@K (Normalized Discounted Cumulative Gain)

**Goal**: Implement NDCG - the most comprehensive ranking quality metric!

**What makes NDCG special:**
- Considers **position** - results at top are more valuable
- Measures **ranking quality** - not just presence/absence
- **Normalized** - always between 0 and 1 (easier to compare)

**What you'll implement:**
1. `dcg_at_k()` - Calculate Discounted Cumulative Gain
2. `ndcg_at_k()` - Normalize DCG by ideal ranking (IDCG)
3. Compare with Precision/Recall to see the difference

**💡 Reference**: Learning Notebook Part 2, NDCG section

In [18]:
# Exercise 10: Implement NDCG@K
import numpy as np

def dcg_at_k(relevances, k):
    """
    Calculate Discounted Cumulative Gain at K.
    """
    # Considerar só os top-k
    rels = np.array(relevances[:k], dtype=float)

    if rels.size == 0:
        return 0.0

    # posições 1,2,3,... (porque o DCG é 1/log2(pos+1) com pos a começar em 1)
    positions = np.arange(1, rels.size + 1)

    # desconto: log2(position + 1)
    discounts = np.log2(positions + 1)

    # DCG = soma( relevance_i / log2(position_i + 1) )
    dcg = np.sum(rels / discounts)
    return float(dcg)


def ndcg_at_k(relevant_ids, search_results, k):
    """
    Calculate Normalized Discounted Cumulative Gain at K.
    """
    relevant_set = set(relevant_ids)

    # Step 1 – relevâncias reais dos resultados (1 se relevante, 0 se não)
    top_k_results = search_results[:k]
    actual_relevances = [1 if doc_id in relevant_set else 0 for doc_id in top_k_results]

    # Step 2 – DCG da ordem real
    dcg = dcg_at_k(actual_relevances, k)

    # Step 3 – relevâncias ideais: todos os relevantes primeiro
    num_relevant = min(len(relevant_set), k)
    ideal_relevances = [1] * num_relevant + [0] * max(0, k - num_relevant)

    # Step 4 – DCG da ordem ideal (IDCG)
    idcg = dcg_at_k(ideal_relevances, k)

    # Step 5 – normalizar
    if idcg == 0:
        return 0.0

    return float(dcg / idcg)



# Test with example data
print("Testing NDCG Implementation:")
print("="*70)

# Ground truth
relevant_movie_ids = [10, 25, 42, 67, 89]

# Search results: some relevant, some not
search_results = [10, 15, 25, 30, 42, 55, 67, 70, 80, 89]
# Relevance pattern: [1, 0, 1, 0, 1, 0, 1, 0, 0, 1]

print(f"Ground truth: {len(relevant_movie_ids)} relevant movies")
print(f"Search results (top 10): {search_results}\n")

# Calculate NDCG at different K values
print("NDCG at different K values:")
print("-"*70)
for k in [1, 3, 5, 10]:
    ndcg = ndcg_at_k(relevant_movie_ids, search_results, k)
    print(f"NDCG@{k}: {ndcg:.3f}")

# Compare all metrics together
print("\n" + "="*70)
print("Comparing All Metrics at K=10:")
print("="*70)
k = 10

prec = precision_at_k(relevant_movie_ids, search_results, k)
rec = recall_at_k(relevant_movie_ids, search_results, k)
ndcg = ndcg_at_k(relevant_movie_ids, search_results, k)
rr = reciprocal_rank(relevant_movie_ids, search_results)

print(f"Precision@{k}: {prec:.3f} - What % of top-{k} are relevant?")
print(f"Recall@{k}:    {rec:.3f} - What % of all relevant found?")
print(f"NDCG@{k}:      {ndcg:.3f} - How good is the ranking?")
print(f"RR:            {rr:.3f} - Position of first relevant result")

print("\n" + "="*70)
print("💡 Key Insights:")
print("="*70)
print("1. NDCG rewards putting relevant results at the TOP")
print("2. NDCG penalizes relevant results buried at the BOTTOM")
print("3. Precision/Recall don't care about position - only presence")
print("4. NDCG is the most comprehensive ranking quality metric!")
print("\n📚 Reference: Learning Notebook Part 2, NDCG section")

Testing NDCG Implementation:
Ground truth: 5 relevant movies
Search results (top 10): [10, 15, 25, 30, 42, 55, 67, 70, 80, 89]

NDCG at different K values:
----------------------------------------------------------------------
NDCG@1: 1.000
NDCG@3: 0.704
NDCG@5: 0.640
NDCG@10: 0.851

Comparing All Metrics at K=10:
Precision@10: 0.500 - What % of top-10 are relevant?
Recall@10:    1.000 - What % of all relevant found?
NDCG@10:      0.851 - How good is the ranking?
RR:            1.000 - Position of first relevant result

💡 Key Insights:
1. NDCG rewards putting relevant results at the TOP
2. NDCG penalizes relevant results buried at the BOTTOM
3. Precision/Recall don't care about position - only presence
4. NDCG is the most comprehensive ranking quality metric!

📚 Reference: Learning Notebook Part 2, NDCG section


i

## Summary: What You've Practiced in Part 2

✅ TF-IDF from Scratch (IDF calculation + full TF-IDF)  
✅ Build TF-IDF Vectors from Scratch (complete vectorization)  
✅ Similarity-Based Search with TF-IDF (cosine similarity)  
✅ Hybrid Search (combining TF-IDF + keyword matching)  
✅ Compare Search Methods (keyword vs TF-IDF vs hybrid)  


**Key Takeaways:**
- TF-IDF improves on BoW by weighting words by importance (TF × IDF)
- IDF measures word rarity across documents (rare words = more informative)
- Cosine similarity measures document similarity regardless of length
- Similarity search is better than keyword search but still keyword-based (not truly semantic)
- Hybrid search combines the best of both: keyword precision + TF-IDF recall
- True semantic search requires embeddings (Class 3!)


**🎉 Congratulations!** You've built a complete search system from scratch! You now understand:
- How to convert text to numbers (BoW, TF-IDF)
- How to measure similarity (cosine similarity)
- How to search and rank documents
- How to combine multiple search strategies (hybrid search)

**Next Steps**:
- Continue with **Exercise Notebook Part 3** to practice using industry tools (NLTK, spaCy, scikit-learn)!
- Review the Learning Notebooks
- Continue with Class 3 for embeddings and semantic search!


